In [ ]:
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, RGCNConv
from models.rgcn import rgcn_link_predict

from utils.path_manage import get_files
from utils.evaluation_metrics import SRR

In [ ]:
triplets, lookup, ASD_dictionary, BCE_dictionary, Edge_list, Edge_features = get_files()
edge_index = torch.tensor(Edge_list, dtype=torch.long)
edge_features = torch.tensor(Edge_features, dtype=torch.float)
edge_type = torch.tensor([x.index(max(x)) for x in Edge_features], dtype=torch.long)

In [ ]:
_no = 20
graph = Data(edge_index=edge_index[:_no].t().contiguous(), edge_attr=edge_features[:_no], edge_type=edge_type[:_no])

In [ ]:
model = rgcn_link_predict(graph=graph, hidden_dim = 10)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
model.parameters

In [ ]:
training_data = torch.LongTensor(triplets[:5])
loss = torch.nn.CrossEntropyLoss()

for epoch in range(100):
    model.train()
    outputs = model.forward(graph.edge_index, graph.edge_type)
    prediction = model.distmult(outputs, training_data)
    training_loss = loss(prediction, training_data[:,2])
    print(training_loss)
    training_loss.backward()
    optimiser.step()

    model.eval()
    outputs = model.forward(graph.edge_index, graph.edge_type)
    prediction = model.distmult(outputs, training_data)
    evaluation_target = training_data[:,2].unsqueeze(0).transpose(0,1)
    MRR = SRR(prediction, evaluation_target) / 5
    print(MRR)


In [ ]:
import numpy as np
fake_edges = np.random.randint(low=0,high=10,size=(10,2))
fake_index = torch.tensor(fake_edges, dtype=torch.long)

In [ ]:

class Net(torch.nn.Module):
    def __init__(self, graph, hidden_dim=100):
        super(Net, self).__init__()
        # Should add embedding layer at start  ----   self.emb_e = torch.nn.Embedding(data.num_nodes, hidden_dim, padding_idx=0)        
        self.emb_rel = torch.nn.Embedding(graph.num_edge_features+1, hidden_dim, padding_idx=0)

        self.conv1 = RGCNConv(in_channels=graph.num_nodes, out_channels=hidden_dim, num_relations=graph.num_edge_features, num_bases=30)
        self.conv2 = RGCNConv(in_channels=hidden_dim, out_channels=hidden_dim, num_relations=graph.num_edge_features, num_bases=30)
             
    def forward(self, edge_index, edge_type):
        # Should add embedding layer to x? to do that you'll need to pass nodes to the forward, which is not done atm.  
        x = F.relu(self.conv1(None, edge_index, edge_type))
        x = self.conv2(x, edge_index, edge_type)
        rel_embedded = self.emb_rel(edge_type)

        return x, rel_embedded

    def distmult(self, out, triplets):
        obj = out[0][triplets[:,0]] 
        rel = out[1][range(len(triplets))]   # use rel = out[1][triplets[0:10,0]] or something similar when your code actually works
        sub = out[0][triplets[:,2]]
        all_scores = torch.mm( (obj*rel), out[0].transpose(0,1) )  # print(obj, rel, sub)
        return all_scores 
        



In [ ]:
out = model(data.edge_index, data.edge_type, data.edge_type)

In [ ]:
score = model.distmult(out, triplets[0:10])

In [ ]:
# you need the score over all tails for BCE!!!
print(score)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
output = loss(all_scores, torch.LongTensor(sample[:,2]))
print(output)

In [ ]:
obj = out[0][sample[:,0]]
# use rel = out[1][triplets[0:10,0]] or something similar when your code actually works
rel = out[1][range(len(sample))]
sub = out[0][sample[:,2]]
# print(obj, rel, sub)
distmult = torch.sum(obj * rel * sub, dim=1)
print(distmult)

In [ ]:
print(sample[:,2])

In [ ]:
out[0].transpose(0,1).shape

In [ ]:
all_scores = torch.mm((obj*rel),out[0].transpose(0,1))

In [ ]:
print(output)

In [ ]:
output.backward()

In [ ]:
print(model.emb_rel.weight.transpose(1, 0))


In [ ]:
data.edge_index

In [ ]:
pred = torch.mm(out[0], )

In [ ]:
triplets[0:2]

In [ ]:
distmult = model.distmult(out, triplets[0:1])
print(distmult)

In [ ]:
model

In [ ]:
for param in model.parameters():
    print(type(param.data), param.size())

In [ ]:
out = model(data.edge_index, data.edge_type)
# this is a call to the forward function!!!

In [ ]:
# okay so your model gives out node embeddings. Now to do a distmult on these bad boys.  

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Entities
from torch_geometric.nn import RGCNConv

name = 'MUTAG'
path = osp.join(
    osp.dirname(osp.realpath('__file__')), '..', 'data', 'Entities', name)
dataset = Entities(path, name)
data = dataset[0]

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = RGCNConv(
            data.num_nodes, 16, dataset.num_relations, num_bases=30)
        self.conv2 = RGCNConv(
            16, dataset.num_classes, dataset.num_relations, num_bases=30)

    def forward(self, edge_index, edge_type, edge_norm):
        x = F.relu(self.conv1(None, edge_index, edge_type))
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
print(model.parameters)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type, data.edge_norm)
    print(out)
    F.nll_loss(out[data.train_idx], data.train_y).backward()
    print(data.train_idx)
    print(out[data.train_idx])
    optimizer.step()